In [1]:
#Importing Required Models for Random_Forest

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [2]:
#Creating Dataframes

#train_df=pd.read_csv(r"C:\Users\TUF\Downloads\Train_Data.csv")
train_df = pd.read_csv("Train_Data.csv")
#test_df =pd.read_csv(r"C:\Users\TUF\Downloads\Test_Data.csv")
test_df = pd.read_csv("Test_Data.csv")
#subm_df=pd.read_csv(r"C:\Users\TUF\Downloads\Sample_Submission.csv")
subm_df = pd.read_csv("Sample_Submission.csv")

In [3]:
#Loading Dataframes

train_df.tail()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
86840,0,tcp,http,SF,216,674,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.01,0.00,0.00,21,0
86841,0,tcp,http,SF,368,1725,0,0,0,0,...,1.00,0.00,0.03,0.05,0.00,0.00,0.13,0.06,20,0
86842,28,tcp,ftp,SF,1483,4152,0,0,0,30,...,0.28,0.04,0.01,0.00,0.00,0.00,0.00,0.00,20,0
86843,0,tcp,smtp,SF,1679,385,0,0,0,0,...,0.16,0.08,0.00,0.00,0.69,0.00,0.01,0.00,18,0
86844,11343,tcp,telnet,SF,3396,16694,0,0,0,0,...,0.18,0.02,0.00,0.00,0.23,0.33,0.00,0.00,18,0


In [4]:
#Inspecting Dataframe

train_df.shape

(86845, 43)

In [5]:
test_df.tail()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,...,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag
21707,0,tcp,http,SF,273,4367,0,0,0,0,...,255,1.00,0.00,0.03,0.01,0.0,0.0,0.0,0.0,21
21708,0,udp,domain_u,SF,45,45,0,0,0,0,...,255,1.00,0.00,0.01,0.00,0.0,0.0,0.0,0.0,18
21709,0,tcp,imap4,S0,0,0,0,0,0,0,...,10,0.04,0.08,0.00,0.00,1.0,1.0,0.0,0.0,20
21710,0,tcp,ftp_data,SF,29,0,0,0,0,0,...,45,0.47,0.07,0.47,0.04,0.0,0.0,0.0,0.0,21
21711,0,tcp,private,S0,0,0,0,0,0,0,...,2,0.01,0.08,0.00,0.00,1.0,1.0,0.0,0.0,21


In [9]:
test_df.shape

(21712, 42)

In [7]:
subm_df.tail()

,attack
21707,0
21708,1
21709,0
21710,0
21711,0


In [10]:
subm_df.shape

(21712, 1)

In [12]:
#Feature Engineering in train_dataframe

train_df.wrongfragment.value_counts()

wrongfragment
0    86845
Name: count, dtype: int64

In [13]:
train_df.numfailedlogins.value_counts()

numfailedlogins
0    86796
1       38
3        5
2        5
4        1
Name: count, dtype: int64

In [32]:
train_df.land.value_counts()

land
0    86841
1        4
Name: count, dtype: int64

In [14]:
train_df.urgent.value_counts()

urgent
0    86843
2        1
1        1
Name: count, dtype: int64

In [15]:
train_df.attack.value_counts()

attack
0    53847
1    32998
Name: count, dtype: int64

In [16]:
train_df.rootshell.value_counts()

rootshell
0    86728
1      117
Name: count, dtype: int64

In [17]:
train_df.isguestlogin.value_counts()

isguestlogin
0    86145
1      700
Name: count, dtype: int64

In [33]:
train_df.ishostlogin.value_counts()

ishostlogin
0    86844
1        1
Name: count, dtype: int64

In [34]:
train_df = train_df.drop(["land", "wrongfragment", "urgent", "numfailedlogins", "rootshell", "numoutboundcmds", "ishostlogin", "isguestlogin"], axis=1)
print("Dropped ")

Dropped 


In [129]:
#Dataframe information and Description

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21712 entries, 0 to 21711
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   duration                21712 non-null  int64  
 1   protocoltype            21712 non-null  object 
 2   service                 21712 non-null  object 
 3   flag                    21712 non-null  object 
 4   srcbytes                21712 non-null  int64  
 5   dstbytes                21712 non-null  int64  
 6   land                    21712 non-null  int64  
 7   wrongfragment           21712 non-null  int64  
 8   urgent                  21712 non-null  int64  
 9   hot                     21712 non-null  int64  
 10  numfailedlogins         21712 non-null  int64  
 11  loggedin                21712 non-null  int64  
 12  numcompromised          21712 non-null  int64  
 13  rootshell               21712 non-null  int64  
 14  suattempted             21712 non-null

In [36]:
train_df.describe()

,duration,srcbytes,dstbytes,hot,loggedin,numcompromised,suattempted,numroot,numfilecreations,numshells,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
count,86845.000000,8.684500e+04,8.684500e+04,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,...,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000,86845.000000
mean,104.166872,8.455707e+03,2.732095e+03,0.144971,0.441142,0.320583,0.001267,0.356382,0.014370,0.000357,...,0.526457,0.050186,0.075810,0.016246,0.324666,0.318934,0.092391,0.091045,20.060522,0.379964
std,1038.273538,3.582141e+05,5.543041e+04,1.837958,0.496527,27.993474,0.048450,28.415210,0.528799,0.020072,...,0.447353,0.103323,0.208388,0.055271,0.464291,0.463506,0.282768,0.281452,1.421425,0.485380
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000
50%,0.000000,4.600000e+01,4.400000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.550000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000
75%,0.000000,2.720000e+02,7.680000e+02,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.070000,0.020000,0.010000,1.000000,1.000000,0.000000,0.000000,21.000000,1.000000
max,40504.000000,8.958152e+07,7.028652e+06,77.000000,1.000000,7479.000000,2.000000,7468.000000,43.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000,1.000000


In [37]:
#Handling Numerical and Categorical features

num=train_df.select_dtypes(include=['int64','float64']).columns
catg=train_df.select_dtypes(include=['object']).columns
len(num)

32

In [41]:
catg = ['protocoltype', 'service', 'flag', "suattempted"]
num = ['duration', 'srcbytes', 'dstbytes', 'hot', 'loggedin', 'numcompromised', 'numroot', 'numfilecreations', 'numshells',
       'numaccessfiles', 'count', 'srvcount', 'serrorrate',
       'srvserrorrate', 'rerrorrate', 'srvrerrorrate', 'samesrvrate',
       'diffsrvrate', 'srvdiffhostrate', 'dsthostcount', 'dsthostsrvcount',
       'dsthostsamesrvrate', 'dsthostdiffsrvrate', 'dsthostsamesrcportrate',
       'dsthostsrvdiffhostrate', 'dsthostserrorrate', 'dsthostsrvserrorrate',
       'dsthostrerrorrate', 'dsthostsrvrerrorrate', 'lastflag']

In [135]:
#Encoding categorical variables using LabelEncoder

encoder = LabelEncoder()

for col in ['protocoltype', 'service', 'flag']:
    train_df[col + '_encoded'] = encoder.fit_transform(train_df[col])
    test_df[col + '_encoded'] = encoder.transform(test_df[col])

In [136]:
#Define categorical columns

cat_columns = ['protocoltype', 'service', 'flag']

#Create a pipeline for preprocessing

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), cat_columns)
    ])

#Append classifier to preprocessing pipelin
e
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

#Training the model

pipeline.fit(train_df.drop('attack', axis=1), train_df['attack'])

#Making predictions on test_df

test_predictions = pipeline.predict(test_df)

In [141]:
#Recreating submission dataframe

subm_df = pd.DataFrame({"attack": test_predictions})


print(subm_df)

       attack
0           1
1           0
2           1
3           1
4           1
...       ...
21707       0
21708       0
21709       1
21710       0
21711       1

[21712 rows x 1 columns]


In [142]:
#subm_df.to_csv(r"C:\Users\TUF\Downloads\Sample_Submission.csv")
subm_df.to_csv("Model_Submission.csv")
print("Submission file created.")

Submission file created.
